In [1]:
import numpy as np
import pandas as pd
import random
from tqdm import tqdm
from bofire.data_models.features.api import CategoricalInput
from bofire.data_models.features.api import ContinuousOutput
from bofire.data_models.objectives.api import MaximizeObjective
from bofire.data_models.domain.api import Inputs, Outputs
from bofire.data_models.domain.api import Domain
from bofire.data_models.strategies.api import QnehviStrategy
from bofire.strategies.mapper import map
from bofire.data_models.surrogates.api import  SingleTaskGPSurrogate, RandomForestSurrogate
from bofire.data_models.surrogates.api import  BotorchSurrogates
from bofire.data_models.surrogates.scaler import ScalerEnum


/opt/homebrew/Caskroom/miniforge/base/envs/bofire/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
x1_options = ['a', 'b', 'c']
x2_options = ['d', 'e', 'f']
x3_options = ['g', 'h', 'i']

# initialize an empty dictionary to store experiment data
experiments_data = {
    'x1': [],
    'x2': [],
    'x3': [],
    'f_1': [],
    'f_2': [],
    'f_3': [],
    'valid_f_1': [],
    'valid_f_2': [],
    'valid_f_3': []
}

# generate 30 experiments
for _ in range(30):
    experiments_data['x1'].append(random.choice(x1_options))
    experiments_data['x2'].append(random.choice(x2_options))
    experiments_data['x3'].append(random.choice(x3_options))
    experiments_data['f_1'].append(round(random.uniform(0.0, 1.0), 2))
    experiments_data['f_2'].append(round(random.uniform(0.0, 1.0), 2))
    experiments_data['f_3'].append(round(random.uniform(0.0, 1.0), 2))
    experiments_data['valid_f_1'].append(1)
    experiments_data['valid_f_2'].append(1)
    experiments_data['valid_f_3'].append(1)

experiments = pd.DataFrame.from_dict(experiments_data)
experiments

,x1,x2,x3,f_1,f_2,f_3,valid_f_1,valid_f_2,valid_f_3
0,a,d,i,0.28,0.91,0.87,1,1,1
1,a,d,h,0.19,0.00,0.69,1,1,1
2,c,f,h,0.41,0.85,0.09,1,1,1
3,c,f,h,0.16,0.39,0.50,1,1,1
4,b,f,g,0.53,0.01,0.70,1,1,1
5,c,f,g,0.63,0.40,0.79,1,1,1
6,b,f,i,0.93,0.71,0.20,1,1,1
7,c,f,i,0.66,0.77,0.19,1,1,1
8,b,f,i,0.61,0.93,0.55,1,1,1
9,c,d,h,0.71,0.87,0.26,1,1,1


In [4]:
# setting up the input space
x1 = CategoricalInput(key = "x1", categories = ['a', 'b', 'c'])
x2 = CategoricalInput(key = "x2", categories = ['d', 'e', 'f'])
x3 = CategoricalInput(key = "x3", categories = ['g', 'h', 'i'])

# setting up the output space
objective = MaximizeObjective(
    w=1.0, 
    bounds= [0.0,1.0],
)
f_1 = ContinuousOutput(key="f_1", objective=objective)
f_2 = ContinuousOutput(key="f_2", objective=objective)
f_3 = ContinuousOutput(key="f_3", objective=objective)

# putting everything together
input_features = Inputs(features = [x1, x2, x3])
output_features = Outputs(features = [f_1, f_2, f_3])

domain = Domain(
    inputs=input_features, 
    outputs=output_features
    )

domain.get_feature_reps_df()

,Type,Description
x1,CategoricalInput,3 categories
x2,CategoricalInput,3 categories
x3,CategoricalInput,3 categories
f_1,ContinuousOutput,ContinuousOutputFeature
f_2,ContinuousOutput,ContinuousOutputFeature
f_3,ContinuousOutput,ContinuousOutputFeature


In [5]:
mobo_strategy_data_model = QnehviStrategy(domain=domain, 
    surrogate_specs=BotorchSurrogates(
        surrogates=[SingleTaskGPSurrogate(inputs=domain.inputs, outputs=domain.outputs.get_by_keys([key]), scaler=ScalerEnum.IDENTITY) for key in ["f_1", "f_2", "f_3"]]))
mobo_strategy = map(mobo_strategy_data_model)
mobo_strategy.tell(experiments = experiments)
mobo_strategy.ask(1)

,x1,x2,x3,f_1_pred,f_2_pred,f_3_pred,f_1_sd,f_2_sd,f_3_sd,f_1_des,f_2_des,f_3_des
0,b,e,i,0.511777,0.653544,0.473413,0.102139,0.128137,0.185293,0.511777,0.653544,0.473413


In [8]:
mobo_strategy_data_model = QnehviStrategy(domain=domain, 
    surrogate_specs=BotorchSurrogates(
        surrogates=[RandomForestSurrogate(inputs=domain.inputs, outputs=domain.outputs.get_by_keys([key]), n_estimators=10) for key in ["f_1", "f_2", "f_3"]]))
mobo_strategy = map(mobo_strategy_data_model)
mobo_strategy.tell(experiments = experiments)
mobo_strategy.ask(1)

,x1,x2,x3,f_1_pred,f_2_pred,f_3_pred,f_1_sd,f_2_sd,f_3_sd,f_1_des,f_2_des,f_3_des
0,b,e,i,0.592,0.7765,0.5695,0.31361,0.146136,0.350035,0.592,0.7765,0.5695


In [9]:
mobo_strategy.model.models[0]._rf

RandomForestRegressor(n_estimators=10)